In [25]:
import numpy as np
import pandas as pd
import geopandas as gp
import os
import subprocess
import math
import shapely as shp
from datetime import datetime
from fiona.crs import from_epsg
import matplotlib.pylab as plt
import re
from pandas.tseries.holiday import get_calendar, HolidayCalendarFactory, GoodFriday
from datetime import datetime

from itertools import compress

np.random.seed(222)

%matplotlib inline

In [3]:
wd = os.getcwd()
os.chdir('../data')
os.getcwd()

'/Users/geoffperrin/Desktop/DLBA/demo_project/data'

In [4]:
#read in detroit feature dataset
city_df = pd.read_csv('CitywideIndicators_20170123.csv')

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
#only keep DLBA Owned properties for the training data
dlba_df = city_df[city_df['Property Ownership'] == 'DLBA Owned']

#remove lots
dlba_df = dlba_df[dlba_df['Property Class'] != 'Residential-Lot']

dlba_df_small = dlba_df.rename(columns=lambda x: x.replace(' ', '_'))
dlba_df_small.head()

,Account_ID,Parcel_ID,Address,Property_Ownership,Property_Class,Program,Current_Status,Current_Status_Date,Neighborhood,Neighborhood_Sales_Median,...,MCM_Institutional_Use,MCM_Public_Use,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping
3,001j000000h4Lo4,21021624.,16272 Lappin,DLBA Owned,Unknown-Lot,Inventory,Intake Review,1/29/2016,Regent Park,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,001j000000Idu4s,21007166.,13064 Elmdale,DLBA Owned,Residential-Structure,Own It Now,Sales Review,5/26/2016,Outer Drive-Hayes,1.40,...,NaN,NaN,Fair,No,NaN,No,NaN,NaN,NaN,Yes
62,001j000000Idu4y,21007448.001,14291 Elmdale,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,4/1/2016,Outer Drive-Hayes,1.40,...,NaN,NaN,Suggest Demolition,Yes,Major,Yes,Open,NaN,NaN,Yes
104,001j000000Idu5O,19005023.,4180 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,NaN,NaN,Poor,No,NaN,Yes,NaN,NaN,NaN,No
116,001j000000Idu5U,19005022.,4174 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,NaN,NaN,Fair,No,NaN,Yes,NaN,NaN,NaN,No


In [18]:
#read in detroit parcel distance to school df
school_df = pd.read_csv('DistanceToSchool_20170123.csv')
school_df_small = school_df[["PARCELNO", "DistanceToSchool_FT"]]
school_df_small.columns = ['Parcel_ID', 'dist_to_school']
school_df_small.head()

,Parcel_ID,dist_to_school
0,16006256.,2196.731029
1,04000103-4,2375.448135
2,04003341.,2442.534185
3,04003339-40,2510.715691
4,04000119.,2341.253487


In [19]:
#merge distance to school to city feature df
dlba_school_df = dlba_df_small.merge(school_df_small, on='Parcel_ID')
dlba_school_df.head()

,Account_ID,Parcel_ID,Address,Property_Ownership,Property_Class,Program,Current_Status,Current_Status_Date,Neighborhood,Neighborhood_Sales_Median,...,MCM_Public_Use,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping,dist_to_school
0,001j000000Idu4s,21007166.,13064 Elmdale,DLBA Owned,Residential-Structure,Own It Now,Sales Review,5/26/2016,Outer Drive-Hayes,1.40,...,NaN,Fair,No,NaN,No,NaN,NaN,NaN,Yes,1235.838510
1,001j000000Idu4y,21007448.001,14291 Elmdale,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,4/1/2016,Outer Drive-Hayes,1.40,...,NaN,Suggest Demolition,Yes,Major,Yes,Open,NaN,NaN,Yes,2823.745964
2,001j000000Idu5O,19005023.,4180 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,NaN,Poor,No,NaN,Yes,NaN,NaN,NaN,No,758.959715
3,001j000000Idu5U,19005022.,4174 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,NaN,Fair,No,NaN,Yes,NaN,NaN,NaN,No,788.673532
4,001j000000Idu5y,18013649.,7729 American,DLBA Owned,Residential-Structure,Inventory,Intake Review,3/18/2015,Midwest,0.84,...,NaN,Poor,No,NaN,Yes,NaN,NaN,NaN,No,4959.561810


### Notes: current status is informative, but program is also informative

### perhaps add a feature that is the number of occupied properties within a buffer of every property - alice will send over parcel_ID lat/long data

In [33]:
len(dlba_school_df)

34014

In [34]:
dlba_school_df[dlba_school_df["Current_Status"] == 'Intake Review']

,Account_ID,Parcel_ID,Address,Property_Ownership,Property_Class,Program,Current_Status,Current_Status_Date,Neighborhood,Neighborhood_Sales_Median,...,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping,dist_to_school,demo_decision,Fire_dummy
4,001j000000Idu5y,18013649.,7729 American,DLBA Owned,Residential-Structure,Inventory,Intake Review,3/18/2015,Midwest,0.84,...,No,NaN,Yes,NaN,NaN,NaN,No,4959.561810,999,0
5,001j000000Idu5Z,19001486.,6628 May,DLBA Owned,Residential-Structure,Inventory,Intake Review,3/18/2015,Airport Sub,0.57,...,No,NaN,No,NaN,NaN,NaN,No,1553.964463,999,0
6,001j000000Idu61,18013683.,7321 American,DLBA Owned,Residential-Structure,Inventory,Intake Review,1/15/2016,Midwest,0.84,...,No,NaN,No,NaN,NaN,NaN,No,3706.193345,999,0
7,001j000000Idu65,19009917.,6480 Crane,DLBA Owned,Residential-Structure,Inventory,Intake Review,3/18/2015,Airport Sub,0.57,...,No,NaN,No,NaN,NaN,NaN,No,1495.080654,999,0
10,001j000000Idu6a,18013672.,7437 American,DLBA Owned,Residential-Structure,Inventory,Intake Review,3/18/2015,Midwest,0.84,...,No,NaN,Yes,NaN,NaN,NaN,No,4129.241243,999,0
11,001j000000Idu6e,18013668.,7465 American,DLBA Owned,Residential-Structure,Inventory,Intake Review,3/18/2015,Midwest,0.84,...,No,NaN,No,NaN,NaN,NaN,Yes,4265.281573,999,0
13,001j000000Idu6V,18013641.,7785 American,DLBA Owned,Residential-Structure,Inventory,Intake Review,1/15/2016,Midwest,0.84,...,No,NaN,No,NaN,NaN,NaN,No,5231.670056,999,0
16,001j000000Idu7i,18009928.,7738 Central,DLBA Owned,Residential-Structure,Inventory,Intake Review,1/15/2016,Midwest,0.84,...,No,NaN,No,NaN,NaN,NaN,No,5286.125693,999,0
18,001j000000Idu8g,22076094.,6546 Rosemont,DLBA Owned,Residential-Structure,Inventory,Intake Review,1/19/2017,Warrendale,2.77,...,No,NaN,No,Secure,NaN,NaN,No,1896.805147,999,0
20,001j000000Idu8N,18009921.,7696 Central,DLBA Owned,Residential-Structure,Inventory,Intake Review,1/15/2016,Midwest,0.84,...,No,NaN,No,NaN,NaN,NaN,No,5076.979421,999,1


In [21]:
#code up demo decision yes / no
demo_decision_dict = {'Demo Awaiting Funding': 1,
        'Demo Pipeline': 1,
        'Demo Contracted': 1,
        'Demo Review': 1,
        'Demolished': 1,
        'Construction Complete': 0,
        'Occupied': 0,
        'Sold': 0,
        'Institutional Partner Coordination': 0,
        'For Sale': 0,
        'Sales Review': 0,
        'Sales Pipeline': 0,
        'Vacant Land': 0,
        'Sales Hold': 0,
        'Under Construction': 0,
        'Lease/ROE/TLA': 0,
        'Warehouse': 0,
        'NPOC Isolation - Do not move!': 999,
        'NSP Quarantine': 999,
        'Reconveyance Complete': 999,
        'Title Problem Identified': 999,
        'DJ Quarantine': 999,
        'PDD Quarantine': 999,
        'Intake Review': 999}

dlba_school_df['demo_decision'] = 0
dlba_school_df['demo_decision'] = [demo_decision_dict[i] for i in dlba_school_df["Current_Status"]]
dlba_school_df.head()

,Account_ID,Parcel_ID,Address,Property_Ownership,Property_Class,Program,Current_Status,Current_Status_Date,Neighborhood,Neighborhood_Sales_Median,...,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping,dist_to_school,demo_decision
0,001j000000Idu4s,21007166.,13064 Elmdale,DLBA Owned,Residential-Structure,Own It Now,Sales Review,5/26/2016,Outer Drive-Hayes,1.40,...,Fair,No,NaN,No,NaN,NaN,NaN,Yes,1235.838510,0
1,001j000000Idu4y,21007448.001,14291 Elmdale,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,4/1/2016,Outer Drive-Hayes,1.40,...,Suggest Demolition,Yes,Major,Yes,Open,NaN,NaN,Yes,2823.745964,1
2,001j000000Idu5O,19005023.,4180 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,Poor,No,NaN,Yes,NaN,NaN,NaN,No,758.959715,1
3,001j000000Idu5U,19005022.,4174 Cadillac,DLBA Owned,Residential-Structure,HHF Demo,Demo Pipeline,3/31/2016,Gratiot Woods,0.59,...,Fair,No,NaN,Yes,NaN,NaN,NaN,No,788.673532,1
4,001j000000Idu5y,18013649.,7729 American,DLBA Owned,Residential-Structure,Inventory,Intake Review,3/18/2015,Midwest,0.84,...,Poor,No,NaN,Yes,NaN,NaN,NaN,No,4959.561810,999


In [29]:
dlba_school_df.columns

Index([u'Account_ID', u'Parcel_ID', u'Address', u'Property_Ownership',
       u'Property_Class', u'Program', u'Current_Status',
       u'Current_Status_Date', u'Neighborhood', u'Neighborhood_Sales_Median',
       u'DFD_Fire:_Occurrence_Date', u'USPS:_Vacancy_Indicated',
       u'USPS:_Last_Update', u'Occupancy_Model:_Probability_Pct',
       u'Occupancy_Model:_Last_Update', u'DNG_Number', u'Foreclosure_Year',
       u'WCT_Foreclosure_Status', u'MCM_Blext_Created_At', u'MCM_Structure',
       u'MCM_Occupancy', u'MCM_Use', u'MCM_Units', u'MCM_Residence_Type',
       u'MCM_Commercial_Use', u'MCM_Industrial_Use', u'MCM_Institutional_Use',
       u'MCM_Public_Use', u'MCM_Condition', u'MCM_Fire_Damage',
       u'MCM_Level_of_Fire_Damage', u'MCM_Boarding_Needed', u'MCM_Trespass',
       u'MCM_Vacant_Lot', u'MCM_Maintained', u'MCM_Dumping', u'dist_to_school',
       u'demo_decision', u'Fire_dummy'],
      dtype='object')

In [32]:
#code NaNs as ... what?
dlba_school_df['Fire_dummy'] = 0
dlba_school_df['Fire_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['DFD_Fire:_Occurrence_Date']]

dlba_school_df['usps_vacant_dummy'] = 0
dlba_school_df['usps_vacant_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['USPS:_Vacancy_Indicated']]

dlba_school_df_small = dlba_school_df.drop(['Account_ID', 'Property_Ownership', 'Property_Class', 'Program', \
                                            'Current_Status', 'Current_Status_Date', 'Neighborhood', 'DNG_Number', \
                                            'WCT_Foreclosure_Status', \
                                            'MCM_Blext_Created_At', 'MCM_Structure', 'MCM_Occupancy',  'MCM_Use',\
                                           'MCM_Units', 'MCM_Residence_Type', 'MCM_Commercial_Use', 'MCM_Industrial_Use',\
                                           'MCM_Institutional_Use', 'MCM_Public_Use', 'MCM_Level_of_Fire_Damage', \
                                            'MCM_Trespass', 'MCM_Vacant_Lot', 'MCM_Maintained', 'DFD_Fire:_Occurrence_Date'], 1)
dlba_school_df_small.head(50)

,Parcel_ID,Address,Neighborhood_Sales_Median,USPS:_Vacancy_Indicated,USPS:_Last_Update,Occupancy_Model:_Probability_Pct,Occupancy_Model:_Last_Update,Foreclosure_Year,MCM_Condition,MCM_Fire_Damage,MCM_Boarding_Needed,MCM_Dumping,dist_to_school,demo_decision,Fire_dummy
0,21007166.,13064 Elmdale,1.40,Yes,1/4/2017,14.54,1/17/2017,2013.0,Fair,No,No,Yes,1235.838510,0,0
1,21007448.001,14291 Elmdale,1.40,Yes,1/4/2017,14.54,1/17/2017,NaN,Suggest Demolition,Yes,Yes,Yes,2823.745964,1,0
2,19005023.,4180 Cadillac,0.59,Yes,1/4/2017,14.54,1/17/2017,2011.0,Poor,No,Yes,No,758.959715,1,0
3,19005022.,4174 Cadillac,0.59,Yes,1/4/2017,14.54,1/17/2017,NaN,Fair,No,Yes,No,788.673532,1,0
4,18013649.,7729 American,0.84,Yes,1/4/2017,14.54,1/17/2017,2010.0,Poor,No,Yes,No,4959.561810,999,0
5,19001486.,6628 May,0.57,No,1/4/2017,58.72,1/17/2017,2013.0,Fair,No,No,No,1553.964463,999,0
6,18013683.,7321 American,0.84,Yes,1/4/2017,14.54,1/17/2017,NaN,Poor,No,No,No,3706.193345,999,0
7,19009917.,6480 Crane,0.57,No,1/4/2017,67.61,1/17/2017,2012.0,Good,No,No,No,1495.080654,999,0
8,18013642.,7777 American,0.84,Yes,1/4/2017,14.54,1/17/2017,2013.0,Poor,No,Yes,No,5197.653491,1,0
9,19009910.,6432 Crane,0.57,Yes,1/4/2017,14.54,1/17/2017,NaN,Good,No,No,No,1733.394976,0,0
